In [27]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from haversine import haversine
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import scipy.stats as stats
import folium
import itertools
import statistics
import string

import warnings
warnings.filterwarnings("ignore")

# Naver Map API

In [28]:
# *-- Geocoding 활용 코드 --*
import json
import urllib
from urllib.request import Request, urlopen

In [29]:
# 주소에 geocoding 적용하는 함수를 작성.
def get_location(loc) :
    # 내 고유값임 (공개안되도록)
    client_id = 'wi1vd6qb31'
    client_secret = 'NWOOthvHMkC11OEqF93vhqXPzsTellMMEq08gtf4'

    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" \
    			+ urllib.parse.quote(loc)
    
    # 주소 변환
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urlopen(request)
    res = response.getcode()
    
    if (res == 200) : # 응답이 정상적으로 완료되면 200을 return
        response_body = response.read().decode('utf-8')
        response_body = json.loads(response_body)
        # print(response_body)
        
        # 주소가 존재할 경우 total count == 1이 반환됨.
        if response_body['meta']['totalCount'] == 1 : 
        	# 위도, 경도 좌표를 받아와서 return해 줌.
            lat = response_body['addresses'][0]['y']
            lon = response_body['addresses'][0]['x']
            return (lon, lat)
        else :
            print('location not exist')
        
    else :
        print('ERROR')

In [30]:
# *-- Directions 5 활용 코드 --*

# option : 탐색옵션 [최대 3개, traoptimal(기본 옵션)]
# trafast 실시간 빠른길
# tracomfort 실시간 편한길
# traoptimal 실시간 최적
# traavoidtoll 무료 우선
# traavoidcaronly 자동차 전용도로 회피 우선
option = 'traoptimal'

def get_optimal_route(start, goal, option=option ) :

    # 내 고유값임 (공개안되도록)
    client_id = 'wi1vd6qb31'
    client_secret = 'NWOOthvHMkC11OEqF93vhqXPzsTellMMEq08gtf4'

    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urllib.request.urlopen(request)
    res = response.getcode()
    
    if (res == 200) :
        response_body = response.read().decode('utf-8')
        return json.loads(response_body)
            
    else :
        print('ERROR')

In [31]:
# 주소에서 위도, 경도 뽑고 dataframe 만드는 함수
def geo_df_coding(addresses):
    geo_list = []
    for address in addresses:
        geo_list.append(get_location(address))
        
    geo_df = pd.DataFrame(geo_list)
    geo_df.columns = ['longitude', 'latitude']
    geo_df = geo_df[['latitude', 'longitude']]
    
    return geo_df, geo_list

In [32]:
# 각 점 간 도로상 거리, 시간 계산해주는 함수
def dist_time_mat(geo_df, geo_list):
    dist_mat = np.zeros((geo_df.shape[0], geo_df.shape[0]))
    time_mat = np.zeros((geo_df.shape[0], geo_df.shape[0]))

    option = 'traoptimal'
    for i in range(geo_df.shape[0]):
        for j in range(geo_df.shape[0]):
            if (i==j):
                dist_mat[i][j] = 0
                time_mat[i][j] = 0
            else:
                results = get_optimal_route(start = geo_list[i], goal = geo_list[j], option=option)

                dist_mat[i][j] = results['route']['traoptimal'][0]['summary']["distance"]
                time_mat[i][j] = results['route']['traoptimal'][0]['summary']["duration"]
                
    return dist_mat, time_mat

In [33]:
# 출발지로부터 각 점에 대한 거리, 시간 계산
def dist_time_start(start, geo_list):
    start_loc = get_location(start)
    
    dist_list = []
    time_list = []

    option = 'traoptimal'

    for i in range(len(geo_list)):
        results = get_optimal_route(start = start_loc, goal = geo_list[i], option=option)

        dist_list.append(results['route']['traoptimal'][0]['summary']["distance"])
        time_list.append(results['route']['traoptimal'][0]['summary']["duration"])
    
    return dist_list, time_list

# Algorithm

In [34]:
# k-means 적용
def k_means(geo_df, k):
    geo_df_k = geo_df.copy()
        
    geo_df_k = geo_df_k.astype('float')
    
    model = KMeans(n_clusters=k, init='k-means++')
    model.fit(geo_df)
    labels = model.predict(geo_df.values)
    geo_df_k['labels'] = labels
    
    return geo_df_k

In [35]:
# 각 그룹에서 출발점으로부터 모든 정점을 거쳤을 때 최단거리 산출하는 함수
# 1. 모든 정점 순서 경우 고려 (permutation)
# 2. 그 순서의 거리의 합을 따져서 최단거리 산출
def min_dist_start(geo_df_k, label, dist_list_start, dist_mat):
    '''
    Input : df, label(군집), dist_list_start(시작-점 거리), dist_mat(점-점 거리)
    Return : 출발점으로부터 군집의 모든 정점을 거쳤을 때 최단거리
    '''
    
    # 모든 point 순서 고려 (permutation)
    permutations = list(itertools.permutations(geo_df_k[geo_df_k['labels']==label].index, sum(geo_df_k['labels']==label)))
    
    # 모든 경우 거리 합 계산 후 최소값 산출
    dist_sum_lst = []
    for i in range(len(permutations)):
        # 출발점(도봉구청)과 첫번째 점의 거리 초기화
        dist_sum =  dist_list_start[permutations[i][0]]
        # 순서대로 합 더해줌
        for j in range(len(permutations[0])-1):
            dist_sum += dist_mat[permutations[i][j], permutations[i][j+1]]
        dist_sum_lst.append(dist_sum)

    # 가장 작은 값 -> 최단거리
    return min(dist_sum_lst)

In [36]:
# 각 그룹에서 출발점으로부터 모든 정점을 거쳤을 때 최단거리인 경우 들린 순서 산출하는 함수
# 1. 모든 정점 순서 경우 고려 (permutation)
# 2. 그 순서의 거리의 합을 따져서 최단거리 산출
def min_dist_start_order(geo_df_k, label, dist_list_start, dist_mat):
    '''
    Input : df, label(군집), dist_list_start(시작-점 거리), dist_mat(점-점 거리)
    Return : 출발점으로부터 군집의 모든 정점을 거쳤을 때 최단거리인 경우 들린 순서
    '''

    # 모든 point 순서 고려 (permutation)
    permutations = list(itertools.permutations(geo_df_k[geo_df_k['labels']==label].index, sum(geo_df_k['labels']==label)))
    
    # 모든 경우 거리 합 계산 후 최소값 산출
    dist_sum_lst = []
    for i in range(len(permutations)):
        # 출발점(도봉구청)과 첫번째 점의 거리 초기화
        dist_sum =  dist_list_start[permutations[i][0]]
        # 순서대로 합 더해줌
        for j in range(len(permutations[0])-1):
            dist_sum += dist_mat[permutations[i][j], permutations[i][j+1]]
        dist_sum_lst.append(dist_sum)

    # 가장 작은 값 -> 최단거리순서
    return permutations[np.argmin(dist_sum_lst)]

In [37]:
# 각 군집의 출발점으로부터 최단거리끼리 차이의 분산
def start_var_diff(geo_df_k, dist_list_start, dist_mat):
    '''
    Input : df, dist_list_start(시작-점 거리), dist_mat(점-점 거리)
    Return : 각 군집의 출발점으로부터 최단거리 모든 조합 차이의 분산
    '''
    min_distance = []
    for i in geo_df_k['labels'].unique():
        min_distance.append(min_dist_start(geo_df_k, i, dist_list_start, dist_mat))

    # k combination 2 : 2개씩 차이의 분산
    combinations = list(itertools.combinations(geo_df_k['labels'].unique(), 2))
    
    diff_list = []
    for i, j in combinations:
        diff = np.abs(min_distance[i] - min_distance[j])
        diff_list.append(float(diff))

    return statistics.variance(diff_list)

In [38]:
# 다른 군집 point 중 해당 군집에 가장 가까운 point를 해당 군집으로 포함시키는 함수
# 중심점에서 가장 가까운 point 해당 군집으로 포함시킴
def near_point_start(geo_df_k, label):
    '''
    Input : df, 변경할 군집
    Return : 군집의 중심에서 다른 군집 point 중 가장 가까운 point 해당 군집으로 변경
    '''
    # 원본 dataframe 변경 방지
    geo_df_k_c = geo_df_k.copy()
    
    center = geo_df_k_c.groupby('labels')[['latitude', 'longitude']].mean().reset_index('labels')[['latitude', 'longitude']]

    # 군집이 i가 아닌 포인트 중 가장 가까운 포인트의 군집(labels)을 i로 변경
    idx = geo_df_k_c[geo_df_k_c['labels']!=label].index

    geo_df_k_c.loc[:, "distance_other_center"] = np.NAN
    for j in idx:
        geo_df_k_c.loc[j, "distance_other_center"] = haversine(geo_df_k_c.loc[j, ['latitude', 'longitude']], center.loc[label,['latitude', 'longitude']])

    geo_df_k_c.loc[np.argmin(geo_df_k_c['distance_other_center']), 'labels'] = label
    del geo_df_k_c['distance_other_center']
    
    return geo_df_k_c

In [39]:
# 모든 군집에 대해 point 추가 진행해보고 가장 분산이 작아지는 경우 선택하고 다음 loop 실행
def min_var_start_algorithm(geo_df_k, dist_list_start, dist_mat):
    '''
    Input : df
    Return : print - 변경될 떄 마다 cnt, var, return - 최종 df
    '''
    geo_df_k_c = geo_df_k.copy()

    cnt = 0
    # print("초기 Var : {}\n".format(start_var_diff(geo_df_k_c)))

    # 기존 군집화, 각 군집 변화시켜보고 var 비교 후 결정
    while(True):
        var_diff_list = []
        var_diff_list.append(start_var_diff(geo_df_k_c, dist_list_start, dist_mat))
        print("Var : {:.5}\n".format(start_var_diff(geo_df_k_c, dist_list_start, dist_mat)))
        
        k = len(geo_df_k_c['labels'].unique())
        for label in range(k):
            geo_df_k_1 = near_point_start(geo_df_k_c, label)
            var_diff_list.append(start_var_diff(geo_df_k_1, dist_list_start, dist_mat))

        if (np.min(var_diff_list) == var_diff_list[0]):
            break
        else:
            geo_df_k_c = near_point_start(geo_df_k_c, np.argmin(var_diff_list)-1)
            cnt += 1
            print("변경 횟수 : {}".format(cnt))
            # print("Var : {}\n".format(start_var_diff(geo_df_k_c)))
            
    print("총 변경 횟수 : {}".format(cnt))
    print("최종 결과 Var : {:.5}".format(start_var_diff(geo_df_k_c, dist_list_start, dist_mat)))
    return geo_df_k_c

# Main

In [15]:
addresses = ['서울시 도봉구 방학로6길 25', 
           '서울시 도봉구 덕릉로63길 19',
           '서울시 도봉구 해등로 133',
           '서울시 도봉구 방학로3길 16',
           '서울시 도봉구 도봉로133길 42',
           '서울시 도봉구 덕릉로59나길 20',
           '서울시 도봉구 해등로16가길 32',
           '서울시 도봉구 우이천로34길 38',
           '서울시 도봉구 노해로 41길 9',
           '서울시 도봉구 도봉로 969',
           '서울특별시 도봉구 마들로 656']

# 출발지 : 도봉산역
start = "서울특별시 도봉구 도봉로 964-40"
start_loc = get_location(start)

geo_df, geo_list = geo_df_coding(addresses)

# 출발지로부터 각 점까지 거리 & 시간
dist_list_start, time_list_start = dist_time_start(start, geo_list)

# 각 점 서로 거리 & 시간
dist_mat, time_mat = dist_time_mat(geo_df, geo_list)

In [44]:
pd.DataFrame(dist_mat)

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,3396.0,1487.0,576.0,916.0,3648.0,3691.0,2821.0,2533.0,4529.0,1892.0
1,3383.0,0.0,2495.0,3006.0,2334.0,441.0,1772.0,2739.0,2519.0,6717.0,3988.0
2,1595.0,1935.0,0.0,1218.0,546.0,2187.0,397.0,2146.0,1858.0,4929.0,2200.0
3,1062.0,2898.0,989.0,0.0,967.0,3150.0,2240.0,2698.0,2410.0,4165.0,1528.0
4,1164.0,2389.0,480.0,678.0,0.0,2641.0,1542.0,2199.0,1911.0,4949.0,2220.0
5,3525.0,142.0,2637.0,3148.0,2476.0,0.0,1914.0,2189.0,1969.0,6859.0,4130.0
6,1767.0,1827.0,1595.0,1390.0,718.0,2079.0,0.0,2038.0,1750.0,5101.0,2372.0
7,2779.0,2436.0,2096.0,2854.0,2182.0,2571.0,2031.0,0.0,288.0,6565.0,3836.0
8,2619.0,2148.0,1808.0,2566.0,1894.0,2283.0,1743.0,288.0,0.0,6277.0,3548.0
9,3625.0,6090.0,4181.0,3419.0,4159.0,6342.0,5432.0,5890.0,5602.0,0.0,3015.0


In [43]:
pd.DataFrame(time_mat)

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,642568.0,405910.0,176449.0,256001.0,686566.0,765973.0,632070.0,562513.0,867828.0,561803.0
1,688974.0,0.0,711828.0,616056.0,526021.0,105687.0,374263.0,698028.0,652537.0,1294124.0,825780.0
2,435057.0,388538.0,0.0,374681.0,157445.0,455429.0,100052.0,544125.0,474568.0,965624.0,582606.0
3,337451.0,629816.0,250310.0,0.0,208529.0,588684.0,871955.0,577637.0,508080.0,799132.0,459931.0
4,337503.0,554837.0,175331.0,196644.0,0.0,621728.0,475898.0,513132.0,443575.0,1002037.0,595544.0
5,704964.0,35906.0,747735.0,635836.0,561928.0,0.0,410170.0,625886.0,580395.0,1333891.0,844938.0
6,478664.0,355539.0,468854.0,386389.0,201052.0,422430.0,0.0,511126.0,441569.0,983286.0,594314.0
7,713661.0,724936.0,662333.0,860186.0,615651.0,917961.0,689098.0,0.0,112042.0,1364969.0,862777.0
8,645487.0,601897.0,539294.0,737147.0,492612.0,805918.0,566059.0,71962.0,0.0,1284370.0,767231.0
9,634891.0,1164571.0,799710.0,592955.0,765186.0,1212872.0,1174225.0,1093489.0,1058382.0,0.0,550027.0


In [16]:
# k_means clustering 적용
geo_df_2 = k_means(geo_df, 2)
geo_df_3 = k_means(geo_df, 3)
geo_df_4 = k_means(geo_df, 4)
geo_df_5 = k_means(geo_df, 5)

geo_df_3_start = geo_df_3.copy()
geo_df_4_start = geo_df_4.copy()

In [17]:
# 각 군집마다 최단거리 산출
print("k=3일 때 클러스터 0의 최단거리 : {}m".format(min_dist_start(geo_df_3_start, 0, dist_list_start, dist_mat)))
print("k=3일 때 클러스터 1의 최단거리 : {}m".format(min_dist_start(geo_df_3_start, 1, dist_list_start, dist_mat)))
print("k=3일 때 클러스터 2의 최단거리 : {}m".format(min_dist_start(geo_df_3_start, 2, dist_list_start, dist_mat)))

k=3일 때 클러스터 0의 최단거리 : 8362.0m
k=3일 때 클러스터 1의 최단거리 : 7232.0m
k=3일 때 클러스터 2의 최단거리 : 1077m


In [18]:
# 각 군집마다 최단거리일 때 방문 순서
print("k=3일 때 클러스터 0의 최단거리 방문 순서 : {}".format(min_dist_start_order(geo_df_3_start, 0, dist_list_start, dist_mat)))
print("k=3일 때 클러스터 1의 최단거리 방문 순서 : {}".format(min_dist_start_order(geo_df_3_start, 1, dist_list_start, dist_mat)))
print("k=3일 때 클러스터 2의 최단거리 방문 순서 : {}".format(min_dist_start_order(geo_df_3_start, 2, dist_list_start, dist_mat)))

k=3일 때 클러스터 0의 최단거리 방문 순서 : (7, 8, 5, 1)
k=3일 때 클러스터 1의 최단거리 방문 순서 : (10, 0, 3, 4, 2, 6)
k=3일 때 클러스터 2의 최단거리 방문 순서 : (9,)


In [19]:
print("K = 3")
geo_df_3_start = min_var_start_algorithm(geo_df_3_start, dist_list_start, dist_mat)

K = 3
Var : 1.0735e+07

변경 횟수 : 1
Var : 1.6983e+06

변경 횟수 : 2
Var : 1.2116e+06

변경 횟수 : 3
Var : 1.0854e+06

총 변경 횟수 : 3
최종 결과 Var : 1.0854e+06


In [20]:
print("K = 4")
geo_df_4_start = min_var_start_algorithm(geo_df_4_start, dist_list_start, dist_mat)

K = 4
Var : 5.7541e+06

변경 횟수 : 1
Var : 6.5479e+05

변경 횟수 : 2
Var : 1.7958e+05

총 변경 횟수 : 2
최종 결과 Var : 1.7958e+05


# Visualization

In [102]:
# 직선으로 시각화
def visual_start_line(geo_df_k_start, start_loc, dist_list_start, dist_mat):
    labels = geo_df_k_start['labels']
    m = folium.Map(location = [start_loc[1], start_loc[0]], zoom_start=14)

    # 각 위치 분류별로 색 다르게
    for i in range(geo_df_k_start.shape[0]):
        if (labels[i] == 0):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'blue')).add_to(m)
        elif (labels[i] == 1):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'red')).add_to(m)
        elif (labels[i] == 2):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'green')).add_to(m)
        elif (labels[i] == 3):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'purple')).add_to(m)
        else:
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'black')).add_to(m)
            
    # 시작 위치
    folium.CircleMarker(location=[start_loc[1], start_loc[0]],
                            radius=20,
                            color='red',
                            fill_color='red',
                            draggable = False).add_to(m)

    # 경로 표시
    colors = ['blue', 'red', 'green', 'purple']

    for i in np.unique(labels):
        orders = min_dist_start_order(geo_df_k_start, i, dist_list_start, dist_mat)
        
        location = []
        for order in orders:
            location.append([geo_df_k_start.loc[order, 'latitude'], geo_df_k_start.loc[order, 'longitude']])
        location.insert(0, [float(start_loc[1]), float(start_loc[0])])
        
        # print(location)

        folium.PolyLine(locations = location, tooltip = 'Polyline', color = colors[i]).add_to(m)

    return m

In [103]:
# K=3
visual_start_line(geo_df_3_start, start_loc, dist_list_start, dist_mat)

In [104]:
# K=4
visual_start_line(geo_df_4_start, start_loc, dist_list_start, dist_mat)

In [99]:
# 진짜 경로로 시각화
def visual_start_driving_line(geo_df_k_start, start_loc, dist_list_start, dist_mat):
    labels = geo_df_k_start['labels']
    m = folium.Map(location = [start_loc[1], start_loc[0]], zoom_start=14)

    # 각 위치 분류별로 색 다르게
    for i in range(geo_df_k_start.shape[0]):
        if (labels[i] == 0):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'blue')).add_to(m)
        elif (labels[i] == 1):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'red')).add_to(m)
        elif (labels[i] == 2):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'green')).add_to(m)
        elif (labels[i] == 3):
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'purple')).add_to(m)
        else:
            folium.Marker([geo_df_k_start.loc[i, 'latitude'], geo_df_k_start.loc[i, 'longitude']], icon=folium.Icon(color = 'black')).add_to(m)
            
    # 중심 위치
    folium.CircleMarker(location=[start_loc[1], start_loc[0]],
                            radius=20,
                            color='red',
                            fill_color='red',
                            draggable = False).add_to(m)

    # 경로 표시
    colors = ['blue', 'red', 'green', 'purple']

    for i in np.unique(labels):
        orders = min_dist_start_order(geo_df_k_start, i, dist_list_start, dist_mat)
        
        location = []
        location.append([float(start_loc[1]), float(start_loc[0])])
        
        results = get_optimal_route(start = start_loc, goal = geo_list[orders[0]], option='traoptimal')
        for t in range(len(results['route']['traoptimal'][0]["path"])):
            location.append([results['route']['traoptimal'][0]["path"][t][1], results['route']['traoptimal'][0]["path"][t][0]])
        
        for j in range(len(orders)-1):
            location.append([geo_list[orders[j]][1], geo_list[orders[j]][0]])
            results = get_optimal_route(start = geo_list[orders[j]], goal = geo_list[orders[j+1]], option='traoptimal')
            for t in range(len(results['route']['traoptimal'][0]["path"])):
                location.append([results['route']['traoptimal'][0]["path"][t][1], results['route']['traoptimal'][0]["path"][t][0]])
        
        location.append([geo_list[orders[-1]][1], geo_list[orders[-1]][0]])

        # print(location)

        folium.PolyLine(locations = location, tooltip = 'Polyline', color = colors[i]).add_to(m)

    return m

In [100]:
# K=3
visual_start_driving_line(geo_df_3_start, start_loc, dist_list_start, dist_mat)

In [101]:
# K=4
visual_start_driving_line(geo_df_4_start, start_loc, dist_list_start, dist_mat)